<a href="https://colab.research.google.com/github/pooriaazami/ASL-dataset/blob/main/ASL_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pooriaazami","key":"91ff20621c61968b71cdbf65dc2a8612"}'}

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d kuzivakwashe/significant-asl-sign-language-alphabet-dataset

100% 3.88G/3.88G [00:48<00:00, 82.4MB/s]
100% 3.88G/3.88G [00:48<00:00, 86.7MB/s]


In [5]:
!unzip significant-asl-sign-language-alphabet-dataset.zip
!rm significant-asl-sign-language-alphabet-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: significant-asl-alphabet-training-set/Training Set/Z/Z190.jpg  
  inflating: significant-asl-alphabet-training-set/Training Set/Z/Z1900.jpg  
  inflating: significant-asl-alphabet-training-set/Training Set/Z/Z1901.jpg  
  inflating: significant-asl-alphabet-training-set/Training Set/Z/Z1902.jpg  
  inflating: significant-asl-alphabet-training-set/Training Set/Z/Z1903.jpg  
  inflating: significant-asl-alphabet-training-set/Training Set/Z/Z1904.jpg  
  inflating: significant-asl-alphabet-training-set/Training Set/Z/Z1905.jpg  
  inflating: significant-asl-alphabet-training-set/Training Set/Z/Z1906.jpg  
  inflating: significant-asl-alphabet-training-set/Training Set/Z/Z1907.jpg  
  inflating: significant-asl-alphabet-training-set/Training Set/Z/Z1908.jpg  
  inflating: significant-asl-alphabet-training-set/Training Set/Z/Z1909.jpg  
  inflating: significant-asl-alphabet-training-set/Training Set/Z/Z191.jpg  
  inflating: si

In [6]:
!rm -rf significant-asl-alphabet-training-set

In [7]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

import numpy as np
import matplotlib.pyplot as plt

In [8]:
dataset = keras.utils.image_dataset_from_directory(
    directory='Training Set',
    labels='inferred',
    label_mode='categorical',
    image_size=(224, 224),
    batch_size=32
).shuffle(32).prefetch(1)

validation_split = .2
validation_count = int(len(dataset) * validation_split)

validation_dataset = dataset.take(validation_count)
training_dataset = dataset.skip(validation_count)

Found 77490 files belonging to 27 classes.


In [9]:
resnet = keras.applications.ResNet50(include_top=False, weights='imagenet')
resnet.trainable = False

94765736/94765736 [==============================] - 0s 0us/step


In [14]:
model = keras.models.Sequential([
    keras.Input(shape=(224, 224, 3)),
    resnet,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(27, activation='softmax')
])

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, None, None, 2048   23587712  
                             )                                   
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dense_3 (Dense)             (None, 128)               12845184  
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dense_5 (Dense)             (None, 27)                3483      
                                                                 
Total params: 36452891 (139.06 MB)
Trainable params: 12865179 (49.08 MB)
Non-trainable params: 23587712 (89.98 MB)
_____

In [16]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.fit(training_dataset, validation_data=validation_dataset, epochs=500, callbacks=[
    keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
    keras.callbacks.ModelCheckpoint(filepath='models', monitor='val_loss', save_best_only=True)
])

Epoch 1/500
1938/1938 [==============================] - 278s 133ms/step - loss: 0.9084 - accuracy: 0.7989 - val_loss: 0.4475 - val_accuracy: 0.8787
Epoch 2/500
1938/1938 [==============================] - 283s 137ms/step - loss: 0.3709 - accuracy: 0.9047 - val_loss: 0.3840 - val_accuracy: 0.9004
Epoch 3/500
1938/1938 [==============================] - 282s 137ms/step - loss: 0.2685 - accuracy: 0.9293 - val_loss: 0.2918 - val_accuracy: 0.9258
Epoch 4/500
1938/1938 [==============================] - 259s 125ms/step - loss: 0.2208 - accuracy: 0.9400 - val_loss: 0.3154 - val_accuracy: 0.9210
Epoch 5/500
1938/1938 [==============================] - 278s 135ms/step - loss: 0.1812 - accuracy: 0.9491 - val_loss: 0.2132 - val_accuracy: 0.9436
Epoch 6/500
1938/1938 [==============================] - 279s 135ms/step - loss: 0.1680 - accuracy: 0.9530 - val_loss: 0.1903 - val_accuracy: 0.9484
Epoch 7/500
1938/1938 [==============================] - 282s 137ms/step - loss: 0.1392 - accuracy: 0.9608

In [21]:
!zip -r models.zip models

  adding: models/ (stored 0%)
  adding: models/saved_model.pb (deflated 92%)
  adding: models/fingerprint.pb (stored 0%)
  adding: models/keras_metadata.pb (deflated 96%)
  adding: models/variables/ (stored 0%)
  adding: models/variables/variables.data-00000-of-00001 (deflated 21%)
  adding: models/variables/variables.index (deflated 78%)
  adding: models/assets/ (stored 0%)


In [ ]:
files.download('models.zip')